# SPRINT 9-PROYECTO

## Descripción del Proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## 1. Examinar la Base de datos

### 1.1 Importar librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split #segmentar entrenamiento y validación
from sklearn.tree import DecisionTreeClassifier #Primer modelo
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier #Segundo modelo
from sklearn.linear_model import LogisticRegression # Tercer modelo
from sklearn.metrics import mean_squared_error

df = pd.read_csv('/datasets/users_behavior.csv')

### 1.2 Examinar Datos

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

Los datos están limpios, no hay duplicados, ni valores ausentes y los tipos de datos por el momento estan bien

## 2 Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba

In [6]:
# Separar los datos en entrenamiento y validación
df_train,df_valid = train_test_split(df, test_size=0.25, random_state=12345)

#Caracteristicas y objetos de entrenamiento y validación de is_ultra
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)
(2410,)
(804, 4)
(804,)


Los tamaños de las tablas que están almacenadas en cuatro variables 

## 3 Investiga la calidad de diferentes modelos cambiando los hiperparámetros.

### 3.1 PRIMER MODELO DecisionTreeClassifier(árbol de decisión)

In [7]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.75
max_depth = 2 : 0.7835820895522388
max_depth = 3 : 0.7885572139303483
max_depth = 4 : 0.7810945273631841
max_depth = 5 : 0.7810945273631841


El mejor resultado es con max_depth = 3, con una precisión de 0.7886.

Esto sugiere que una profundidad de 3 es un buen punto de equilibrio entre modelo simple y precisión. Ir más profundo podría llevar al sobreajuste.

### 3.2 SEGUNDO MODELO RandomForestClassifier (Bosque Aleatorio)

In [ ]:
best_score = 0
best_est = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    model.fit(features_train,target_train)
    score = model.score(features_valid,target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

El mejor modelo de Random Forest se logró con 4 árboles (n_estimators = 4).

Su exactitud en el conjunto de validación fue de 0.7886.

### 3.3 TERCER MODELO LogisticRegression (Regresión Logística)

In [ ]:
model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train,target_train) # entrenaR el modelo en el conjunto de entrenamiento
score_train = model.score(features_train,target_train) # calcular la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid,target_valid) # calcular la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

El modelo no se está sobreajustando (la precisión en validación es similar o 
ligeramente mejor que en entrenamiento).

Tiene menor precisión en validación comparado con los modelos anteriores (Árbol y Bosque).

## CONCLUSIÓN

Se compararon tres modelos: árbol de decisión, bosque aleatorio y regresión logística.

El bosque aleatorio fue el más preciso y estable (accuracy = 0.7886) con solo 4 árboles, superando ligeramente al árbol de decisión y a la regresión logística (accuracy = 0.7537).

Recomendación: Usar el RandomForestClassifier, ya que combina buena precisión y menor riesgo de sobreajuste.